In [ ]:
!pip install -q transformers datasets accelerate evaluate torch


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import math


In [ ]:
dataset = load_dataset("yelp_review_full")
dataset


## Dataset Description

The Yelp Review Full dataset is obtained from the Hugging Face library.
It consists of customer-written reviews in natural language.
The dataset is used to fine-tune a Small Language Model on real-world textual data.


In [ ]:
model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


## Model Selection

DistilGPT-2 is selected as the Small Language Model for this task.
It has approximately 82 million parameters, which is well below the 3 billion parameter limit.
The model is lightweight and suitable for fine-tuning using Google Colab.


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(2000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(500))

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)


## Data Subsampling

The original dataset is very large, so a smaller subset is selected for training and evaluation.
This reduces computational cost while still demonstrating effective fine-tuning of the language model.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
    fp16=True
)


## Training Configuration

In this step, the training parameters for fine-tuning the model are defined.
These parameters control the learning process such as learning rate, batch size, number of epochs, and evaluation method.
Proper configuration helps the model train efficiently and produce better results.


In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator
)

trainer.train()


## Model Fine-tuning

The Small Language Model was successfully fine-tuned on the selected dataset.
Training was performed for two epochs, and both training and validation loss values decreased, indicating effective learning.


In [ ]:
eval_results = trainer.evaluate()
eval_results


## Model Evaluation

The fine-tuned model was evaluated on the test dataset.
Evaluation loss was used as the primary metric to measure the model’s performance on unseen data.


In [ ]:
import math

perplexity = math.exp(eval_results["eval_loss"])
perplexity



## Perplexity

Perplexity was calculated from the evaluation loss.
A lower perplexity value indicates that the model is better at predicting the next word in a sequence.


In [ ]:
input_text = "The restaurant was"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_length=50,
    temperature=0.7,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


## Text Generation

After fine-tuning, the model was used to generate text based on a given prompt.
The generated output shows that the model learned meaningful language patterns from the training data.


## Conclusion

In this task, a Small Language Model was successfully fine-tuned on a text dataset from Hugging Face.
The model showed improved performance after training, as reflected by evaluation loss, perplexity, and generated text.
This experiment demonstrates the complete workflow of fine-tuning and evaluating a language model.
